In [6]:
import pandas as pd
import numpy as np
import operator
import pprint
tree={}

In [7]:
def entropy(class_distribution):
    return np.sum(np.log(1/class_distribution) * class_distribution)

In [8]:
def information_gain(entropy_levelbefore, entropy_oflevel, data_points_in_each_node):
    return entropy_levelbefore - (np.sum((entropy_oflevel*data_points_in_each_node)/np.sum(data_points_in_each_node)))

In [9]:
def find_subtree(dataset,feature,side):
    if side == 'left':
        f= feature.split(" ")
        l= dataset[dataset[f[0]] >= float(f[1])]
        return l
    else:
        f= feature.split(" ")
        l= dataset[dataset[f[0]] < float(f[1])]
        return l

In [10]:
def select_feature(dataset,prev_entropy,entropy_cache):
    en={}
    info={}
    cols=dataset.columns
    cols=list(cols)
    del cols[-1]
    
    for feature in cols:
        for s in set(list(dataset[feature])):
                left= dataset[dataset[feature] >= s]
                right=dataset[dataset[feature] < s]
                
                left_distribution =(np.asarray(list(left['class'].value_counts())))/len(left['class'])
                right_distribution =(np.asarray(list(right['class'].value_counts())))/len(right['class'])
                
                if str(left_distribution) not in entropy_cache:
                    en[feature+" left"] = entropy(left_distribution)
                    entropy_cache[str(left_distribution)]=en[feature+" left"]
                else:
                    en[feature+" left"] =entropy_cache[str(left_distribution)]

                if str(right_distribution) not in entropy_cache:
                    en[feature+" right"] = entropy(right_distribution)
                    entropy_cache[str(right_distribution)]=en[feature+" right"]
                else:
                    en[feature+" right"]=entropy_cache[str(right_distribution)]
               
                info[str(feature)+" "+str(s)]=information_gain(prev_entropy,np.asarray([float(en[feature+" left"]),float(en[feature+" right"])]),np.asarray([len(left),len(right)]))
    best_feature=max(info.items(), key=operator.itemgetter(1))[0]
    return best_feature

In [11]:
data = open('spambase.data').read().split("\n")
data = [i.split(",") for i in (data)]
column = open('features','r').read().split("\n")
column =[i.strip() for i in column]
dataset =  pd.DataFrame(data, columns=column)
dataset=dataset.convert_objects(convert_numeric=True)
dataset=dataset.dropna()
columnmax = [max(dataset[i]) for i in dataset.columns]
columnmin = [min(dataset[i]) for i in dataset.columns]
for i,j in enumerate(dataset.columns):
    dataset[j] = (dataset[j] - columnmin[i])/columnmax[i]
dataset['class']= dataset['class'].astype(int)
# training_dataset=dataset.sample(frac=1)[:-1000]
# test_dataset=dataset.sample(frac=1)[-1000:]

/Users/shaivalpatel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


In [12]:

def training(dataset,depth,tree,entropy_cache={}):
    current_node_entropy = entropy((np.asarray(list(dataset['class'].value_counts())))/len(dataset['class']))
    if (current_node_entropy>=0.25 and depth !=0):
        
            current_node_entropy = entropy((np.asarray(list(dataset['class'].value_counts())))/len(dataset['class']))
            best_feature = select_feature(dataset,current_node_entropy,entropy_cache)
            print(best_feature)
            left_subtree= find_subtree(dataset,best_feature,"left")
            right_subtree=find_subtree(dataset,best_feature,"right")
            if not left_subtree.empty:
                tree["> "+str(best_feature)]=training(left_subtree,depth-1,{},entropy_cache)
            if not right_subtree.empty:
                tree["< "+str(best_feature)]=training(right_subtree,depth-1,{},entropy_cache)
            return tree
    else:
        return (dataset['class'].value_counts())

In [13]:
r 

NameError: name 'r' is not defined

In [ ]:
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(tree)

In [22]:
def testing(row,tree,depth):
 
    if  depth !=0 and type(tree)==dict :
        for key in tree.keys():
            operator = key.split(" ")[0]
            feature =key.split(" ")[1]
            threshold=key.split(" ")[2]
            row_feature = row[feature]
            if operator == '<':
                if (row_feature)< float(threshold):
                    result=testing(row,tree[key],depth-1)
                    return result
            else:
                if (row_feature)>= float(threshold):
                    result=testing(row,tree[key],depth-1)
                    return result
    else:
        
        if 0 not in tree:
            return 1
        elif 1 not in tree:
            return 0
        else:
            if tree[0]>tree[1]:
                return 0
            else:
                return 1

In [14]:
prediction=[]
for index,row in test_dataset.iterrows():
    prediction.append(testing(row,tree,10))

NameError: name 'test_dataset' is not defined

In [15]:
def accuracy(prediction,test_dataset):
    test=list(test_dataset['class'])
    ac = zip(prediction,test)
    right =0
    for k,v in ac:
        if k ==v:
            right+=1
    return right/len(prediction)

In [16]:
acc = accuracy(prediction,test_dataset)

NameError: name 'test_dataset' is not defined

In [17]:
acc

NameError: name 'acc' is not defined

In [23]:
%%time
ac=[]
dataset = dataset.sample(frac=1)
for i in range(0,len(dataset),920):
    print(i)
    if i+920<len(dataset):
        tree={}
        
        training_dataset=pd.concat([dataset[i+920:],dataset[0:i]])
        test_dataset=dataset[i:i+920]
        tree=training(training_dataset,12,tree)
        prediction=[]
        for index,row in test_dataset.iterrows():
            prediction.append(testing(row,tree,12))
        acc = accuracy(prediction,test_dataset)
        ac.append(acc)
print(np.sum(np.asarray(ac))/5)

0
char_freq_! 0.0024632058624299525
char_freq_$ 0.0021655838747293025
word_freq_remove 0.016506189821182946
capital_run_length_average 0.0015918367346938775
char_freq_! 0.01573372744627132
word_freq_edu 0.05442176870748299
word_freq_85 0.0075
word_freq_pm 0.019801980198019802
word_freq_your 0.08100810081008102
word_freq_hp 0.02112337974075852
word_freq_all 0.04313725490196079
capital_run_length_total 0.004418912947414936
word_freq_make 0.35903083700440525
word_freq_you 0.14400000000000002
word_freq_will 0.06101344364012409
word_freq_re 0.005135387488328665
word_freq_free 0.02
char_freq_( 0.012305168170631663
word_freq_free 0.0115
char_freq_! 0.01613399839891619
word_freq_free 0.014499999999999999
word_freq_free 0.0715
capital_run_length_total 0.0036613850135723755
word_freq_all 0.2823529411764706
word_freq_report 0.023
word_freq_email 0.13971397139713973
word_freq_order 0.12927756653992398
char_freq_! 0.025063119650224768
capital_run_length_total 0.0008837825894829872
word_freq_you 0.1

In [19]:
import multiprocessing as mp

In [4]:
mp.cpu_count()

4